In [ ]:
import pickle
from himpy.histogram import operations, expressionOperations
from himpy.executor import Parser, Evaluator
from himpy.utils import E
from utils.search_engine import SearchEngine
from utils.plot.plotly_plot import show_rank_images

In [ ]:
parser = Parser()
Ec_green        = E("e1+e2+e3+e4+e5+e6+e7+e8+e9+e10+e11+e12+e13+e14+e15+e16+e17+e18+e19+e20")
Ec_yellow_green = E("e2+e3+e21+e22+e23+e24+e25+e26+e27+e28+e29+e30")
Ec_red          = E("e31+e32+e33+e34+e35+e36+e37+e38+e39+e40")
Ec_rose         = E("e32+e35+e36+e39+e40")
Ecs = [
    ("green", Ec_green),
    ("yellow_green", Ec_yellow_green),
    ("red", Ec_red),
    ("rose", Ec_rose)
]
high_level_elements = { name: parser.parse_set(Ec.value) for name, Ec in Ecs}

In [ ]:
with open("hists.pkcl", "rb") as f:
    hists = pickle.load(f)
with open("images.pkcl", "rb") as f:
    images = pickle.load(f)

In [ ]:
evaluator = Evaluator(operations, expressionOperations, high_level_elements=high_level_elements)
search_engine_default = SearchEngine(hists, parser, evaluator, use_index=False)
search_engine_inverted_index = SearchEngine(hists, parser, evaluator, use_index=True)

In [ ]:
TOP_N = 20

E1 = E("green")
E2 = E("yellow_green")
E3 = E("red")
E4 = E("rose")
E5 = E("any")

query = E1 * E('e1+e2+e3')

In [ ]:
ranked_images = search_engine_inverted_index.retrieve(query, top_n=1000)
print("Total retrieved images:", len(ranked_images))
ranked_images[:5]

In [ ]:
fig = show_rank_images(images, ranked_images, limit=TOP_N, 
                       title="Top {}: <b>{}</b>".format(TOP_N, query.value))
fig.show()

In [ ]:
ranked_images2 = search_engine_default.retrieve(query, top_n=1000)
print("Total retrieved images:", len(ranked_images2))
ranked_images2[:5]

In [ ]:
fig = show_rank_images(images, ranked_images2, limit=TOP_N, 
                       title="Top {}: <b>{}</b>".format(TOP_N, query.value))
fig.show()

In [ ]:
with open("inverted_ranks.txt", "w") as f:
    for id, score in ranked_images:
        green = evaluator.eval(["green"], hists[id][1]).sum() * 25
        yellow_green = evaluator.eval(["yellow_green"], hists[id][1]).sum() * 25
        red = evaluator.eval(["red"], hists[id][1]).sum() * 25
        rose = evaluator.eval(["rose"], hists[id][1]).sum() * 25
        f.write(f"id: {id:>3}, score: {round(score, 4):>6}, green: {round(green):>2}, yellow: {round(yellow_green):>2}, red: {round(red):>2}, rose: {round(rose):>2}\n")
with open("brute_ranks.txt", "w") as f:
    for id, score in ranked_images2:
        green = evaluator.eval(["green"], hists[id][1]).sum() * 25
        yellow_green = evaluator.eval(["yellow_green"], hists[id][1]).sum() * 25
        red = evaluator.eval(["red"], hists[id][1]).sum() * 25
        rose = evaluator.eval(["rose"], hists[id][1]).sum() * 25
        f.write(f"id: {id:>3}, score: {round(score, 4):>6}, green: {round(green):>2}, yellow: {round(yellow_green):>2}, red: {round(red):>2}, rose: {round(rose):>2}\n")